In [1]:
import itertools
import os

import time
import traceback

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
from scipy.optimize import least_squares

from spiketrain.sptr import SpikeTrains

from actxtimescale.constants import DATA_FOLDER, DATAFRAMES_FOLDER
from actxtimescale.data.celldata import CellData
from actxtimescale.data.stimdata import StimData
from actxtimescale.dfmp import DataFrameMP
from actxtimescale.dichotomized_gaussian import DichotomizedGaussian
from actxtimescale.funs import get_autocorrelation, get_autocorr_after_stimulus, fit_exponential_fixed_offset, estimate_surrogate, bias_variance, neuron_autocor_layout, plot_autocorrelation_summary
from actxtimescale.utils import list_columns_as_rows

%load_ext autoreload
%autoreload 2

### contents

[get_autocorrelation](#autocorrelation) 

[fit_exp_fixed_offset](#fit_exp_fixed_offset) 

[single_file_all](#single_file_all)

[surrogate_taus](#surrogate_taus)

[bias_var_tau](#bias_var_tau)

### get autocorrelation <a name="autocorrelation"></a>  
[contents](#contents)

In [ ]:
dtbins = [20]

folders = []
for hem_folder in ['ACx_data_1/ACxCalyx/', 'ACx_data_2/ACxCalyx/', 'ACx_data_3/ACxCalyx/', 
                   'ACx_data_1/ACxThelo/', 'ACx_data_2/ACxThelo/', 'ACx_data_3/ACxThelo/']:
    folders += [hem_folder + folder for folder in os.listdir(DATA_FOLDER + hem_folder)]

l = list(itertools.product(folders, dtbins))
df = pd.DataFrame(l, columns=['cell', 'dtbin'])
df['set'] = df.cell.str[9].astype(int)

df = df.reset_index(drop=True)
df['hemisphere'] = df['cell'].apply(lambda c: 'calyx' if 'Calyx' in c else 'thelo')

In [ ]:
### This cell computes the autocorrelation for the neurons and outputs a dataframe

import warnings
 
warnings.filterwarnings('ignore')

b = None
t0bin, tfbin, biased = 100, 1640, False

df['t0bin'] = t0bin
df['tfbin'] = tfbin
df['biased'] = int(biased)
df['hemisphere'] = df['cell'].apply(lambda c: 'calyx' if 'Calyx' in c else 'thelo')

columns = ['dt', 'n_spikes', 'n_trials', 'mean_spikes', 'sd_spikes', 'mean_isi', 'sd_isi', 'autocorr', 
           'sd_autocorr', 'mse_mean', 'mse_offset']
dfd = DataFrameMP(df=df.iloc[:6])

time0 = time.time()
dfd = dfd.join(dfd.apply(get_autocorrelation, kwargs=dict(root_folder=DATA_FOLDER), columns=columns, processes=128))

df_name = 'df_raw_autocorrelation.json'
path = DATAFRAMES_FOLDER + df_name

if not(df_name in os.listdir(DATAFRAMES_FOLDER)):
    dfd.df.to_json(path, double_precision=15)
else:
    df_autocorrelation = pd.read_json(path, dtype={'mse': float})
    df_autocorrelation = df_autocorrelation.sort_index()
    dfd.df.index = np.arange(len(dfd.df)) + len(df_autocorrelation)
    df_autocorrelation = df_autocorrelation.append(dfd.df)
    df_autocorrelation = df_autocorrelation.drop_duplicates(['cell', 'dtbin', 't0bin', 'tfbin', 'biased'])
    df_autocorrelation.to_json(path, double_precision=15)

timef = time.time()

print('took', (timef - time0) / 60, 'minutes')

In [ ]:
dfd.df

In [ ]:
plt.plot(dfd.df.loc[2, 'autocorr'])

### fit exponential fixed offset <a name="fit_exp_fixed_offset"></a>  
[contents](#contents)

In [ ]:
### This cell fits an exponential function to the computed autocorrelations

import warnings
 
warnings.filterwarnings('ignore')

tf_autocorr = 760

df_autocorrelation = pd.read_json(DATAFRAMES_FOLDER + 'df_raw_autocorrelation.json', dtype={'mse': float})

df = df_autocorrelation[~df_autocorrelation['autocorr'].isna()].sort_index().reset_index().copy()
df = df[(df.dtbin.isin([20])) & (df.t0bin.isin([100])) & (df.tfbin.isin([1640])) & (df.biased.isin([0, 1]))]
# df['tau_hemisphere'] = df['hemisphere'].map(df.groupby('hemisphere')['tau'].median())

df['tf_autocorr'] = tf_autocorr
df['fit_offset'] = False
df['random_state'] = np.random.randint(1e8, size=len(df))

A0s = [1e-4, 1e-3, 1e-2, 1e-1, 1e0]
tau0s = [40, 80, 160, 320, 640]
offset0s = [5e-5, 1e-3, 5e-2]
n_init = 40 # gives a total of n_init * len(A0s) initializations
bounds = ([0, 5e0], [0, np.inf])
# bounds = ([-np.inf, np.inf], [-np.inf, np.inf])

time0 = time.time()

columns = ['t0_autocorr', 'Abin_est', 'tau_est', 'offset_bin_est', 'mse_lsq']
dfd = DataFrameMP(df=df.iloc[:].copy())
dfd = dfd.join(dfd.apply(fit_exponential_fixed_offset, kwargs=dict(tau0s=tau0s, A0s=A0s, offset0s=offset0s, 
                                                                   n_init=n_init, bounds=bounds, uniform=True), 
                         processes=128, columns=columns))
dfd.df = dfd.df.drop(['autocorr', 'sd_autocorr', 'mse_mean', 'mse_offset', 'random_state'], axis=1)

df_name = 'df_raw_autocorrelation_tau.json'
path = DATAFRAMES_FOLDER + df_name

if not(df_name in os.listdir(DATAFRAMES_FOLDER)):
    dfd.df.to_json(path, double_precision=15)
else:
    df_autocorrelation_tau = pd.read_json(path, dtype={'mse': float})
    df_autocorrelation_tau = df_autocorrelation_tau.append(dfd.df, ignore_index=True)
    df_autocorrelation_tau.to_json(path, double_precision=15)

timef = time.time()

print('took', (timef - time0) / 60, 'minutes')

In [ ]:
dfd.df

### estimate surrogate taus <a name="surrogate_taus"></a>  
[contents](#contents)

In [2]:
### Given the exponential fits, this cell generates DG models with exponential autocorrelations, generate samples and refits exponentials to obtain new tau estimations

import warnings
 
warnings.filterwarnings('ignore')

df_autocorr_tau = pd.read_json(DATAFRAMES_FOLDER + 'df_raw_autocorrelation_tau.json', dtype={'mse': float})

dtdg = 1
A0s = [1e-4, 1e-3, 1e-2, 1e-1, 1e0]
tau0s = [40, 80, 160, 320, 640]
offset0s = [5e-5, 1e-3, 5e-2]
n_init = 40
bounds = ([0, 5e0], [0, np.inf])
n = 2 
shared_hem_tau = False

df = df_autocorr_tau.copy()
df['logtau_hemisphere'] = df['hemisphere'].map(df.groupby('hemisphere')['tau_est'].apply(lambda tau: np.nanmedian(np.log(tau))))
df = df[~df.tau_est.isna()]
df = df[df.tf_autocorr.isin([760]) & (df.tfbin.isin([1640])) & (df.t0bin == 100) & (df.biased.isin([0, 1])) & \
        df.dtbin.isin([20]) & ~(df.fit_offset)]
df['shared_hem_tau'] = shared_hem_tau

df = df.sort_values(['index', 'tf_autocorr', 'mse_lsq'], ascending=True).drop_duplicates(['index', 'tf_autocorr'])

df = df.drop('mse_lsq', axis=1)
df = df.rename({'Abin_est': 'Abin', 'tau_est': 'tau_true', 'offset_bin_est': 'offset_bin'}, axis=1)

df = pd.concat([df] * n, ignore_index=True)
df['random_state'] = np.random.randint(1e8, size=len(df))

print('n_pars', len(df))

columns_out = ['fr_est', 'mu_est', 'var_est', 'raw_autocorr', 'Abin_dg', 'tau_dg', 'offset_bin_dg', 'mse', 'max_error', 'cholesky']

dfd = DataFrameMP(df=df.iloc[:])

time0 = time.time()
dfd = dfd.join(dfd.apply(estimate_surrogate, kwargs=dict(dtdg=dtdg, tau0s=tau0s, A0s=A0s, offset0s=offset0s, 
                                                                   n_init=n_init, bounds=bounds, uniform=True), 
                         processes=4, columns=columns_out))
dfd.df = list_columns_as_rows(dfd.df, drop_index=True)
dfd.df = dfd.df.drop(['Abin', 'random_state'], axis=1)

df_name = 'df_tau_bias_variance_data_shared_tau.json'
path = DATAFRAMES_FOLDER + df_name

if not(df_name in os.listdir(DATAFRAMES_FOLDER)):
    dfd.df.to_json(path, double_precision=15)
else:
    df_dg = pd.read_json(path, dtype={'mse': float})
    df_dg = df_dg.append(dfd.df, ignore_index=True)
    df_dg.to_json(path, double_precision=15)

timef = time.time()

print('took', (timef - time0) / 60, 'minutes')

n_pars 12
took 0.1315383513768514 minutes


In [3]:
dfd.df

,index,cell,dtbin,set,hemisphere,t0bin,tfbin,biased,dt,n_spikes,...,fr_est,mu_est,var_est,raw_autocorr,Abin_dg,tau_dg,offset_bin_dg,mse,max_error,cholesky
0,0,ACx_data_1/ACxCalyx/20200107-009-003,20,1,calyx,100,1640,0,0.1,231,...,0.635823,0.012716,0.013204,"[0.013365800865800866, 0.0010416666666666667, ...",0.000635,310.375862,NaN,3.448232e-08,1.486215e-09,1
1,1,ACx_data_1/ACxCalyx/20170909-007_file001,20,1,calyx,100,1640,0,0.1,429,...,2.047258,0.040945,0.042155,"[0.04383116883116884, 0.002832602339181286, 0....",0.001635,197663.079886,NaN,3.612760e-07,1.122309e-08,1
2,2,ACx_data_1/ACxCalyx/20080930-002,20,1,calyx,100,1640,0,0.1,467,...,2.087843,0.041757,0.042899,"[0.044642857142857144, 0.004294590643274854, 0...",0.001855,982.074109,NaN,6.048438e-07,1.300343e-08,1
3,3,ACx_data_1/ACxCalyx/20200108-008-002,20,1,calyx,100,1640,0,0.1,1220,...,5.519481,0.110390,0.111371,"[0.12355699855699855, 0.013432017543859648, 0....",0.002078,157.338007,NaN,1.542532e-06,6.274002e-08,1
4,4,ACx_data_1/ACxCalyx/20191010-005,20,1,calyx,100,1640,0,0.1,170,...,0.372387,0.007448,0.010092,"[0.01014755853465531, 0.0008960573476702509, 0...",0.002484,18.193218,NaN,7.034704e-09,6.423665e-10,1
5,5,ACx_data_1/ACxCalyx/20100428-001,20,1,calyx,100,1640,0,0.1,228,...,1.358826,0.027177,0.028121,"[0.02886002886002886, 0.0018274853801169592, 0...",0.001179,257.881222,NaN,2.047026e-07,6.124284e-09,1
6,0,ACx_data_1/ACxCalyx/20200107-009-003,20,1,calyx,100,1640,0,0.1,231,...,0.749459,0.014989,0.015306,"[0.015530303030303031, 0.000986842105263158, 0...",0.001093,262.489240,NaN,1.651599e-08,1.486215e-09,1
7,1,ACx_data_1/ACxCalyx/20170909-007_file001,20,1,calyx,100,1640,0,0.1,429,...,2.187049,0.043741,0.045074,"[0.046987734487734495, 0.0028326023391812864, ...",0.001480,2046.329406,NaN,3.716062e-07,1.122309e-08,1
8,2,ACx_data_1/ACxCalyx/20080930-002,20,1,calyx,100,1640,0,0.1,467,...,1.839827,0.036797,0.037607,"[0.03896103896103896, 0.0022843567251461987, 0...",0.000446,1290.360975,NaN,2.024132e-07,1.300343e-08,1
9,3,ACx_data_1/ACxCalyx/20200108-008-002,20,1,calyx,100,1640,0,0.1,1220,...,5.384199,0.107684,0.107452,"[0.11904761904761904, 0.012701023391812866, 0....",0.001694,204.032708,NaN,1.450525e-06,6.274002e-08,1
